## Molecules

Molecules will render a 3D charge field model of molecular matter 
from atoms selected by the user.  

That's the plan anyway. At first I'll try making atoms. Molecules 
is Version 2 of Atom Builder (AB), replacing AB's 19 i-loop slots 
with a 19 slot csv data file. The expected faster renderings and 
efficiencies gained should allow us to build multiple atoms in 
molecular configurations. 

Update: 2/24/22. The Periodic table and Slotlayout diagram are 
reconfigured to read the Cr6-Elements.csv datafile. Added the tab 
enclosure which shows 1. a non-interactive Periodic table, or 2. a 
second SL diagram. Added Atom Builder's 3D modeling cell's: Groups(), 
geometries, materials and functions necessary in order to begin 
making atoms. 

Molecules is intended to introduce Charge field theory - see the 
notebook's end cell's source references.                                          

In [1]:
# The imports necessary to create the 3D modeling 
# and gui controls.
from pythreejs import *
from IPython.display import display
from ipywidgets import HBox, VBox, Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan
from random import random , randint , seed

In [2]:
# A few values used
phi = (1+sqrt(5))/2  # calculating phi-based-pi, piG.
piG = 4/sqrt(phi)    # piG is used instead of pi.
pRadius = 0.05 # proton radius, used by all atomic distances
caroSecs = 5.0 # Problem Here # time in secs for the carousel to make one rotation

In [3]:
# For display, sizing the view, camera and adding lights.
view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 0, 1], aspect=view_width/view_height)
#camera = PerspectiveCamera(position=[5, 0, 0], up=[1, 0, 0], aspect=view_width/view_height)
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 1, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [4]:
# For working with a dataFrame.
import pandas as pd
import numpy as np
# Sizing the users' local display of tabular data if needed.
#pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)
filename = '../data/Cr6-Elements.csv'   
df_raw = pd.read_csv(filename) 
# Uncomment df_raw or df_raw.info() to view
# df_raw
# df_raw.info()

# Make changes to the dataFrame created by pd.read_csv(filename)'s 
# method defaults. The data type constrains how you may work with 
# the data.

# Remove the .csv slotLayout diags from dataFrame, they're
# for human viewers, not for program readable data values.
# This programm creates it own SL diagram. 
df = df_raw.drop(columns = ['SlotLayout', 'SL1', 'SL2', 'SL3', 'SL4'])

# Duplicating "AtomicNumber", one as a string,the other an int.
df["AtomicSlotGrp"] = df["AtomicNumber"].copy()
# df["AtomicNumber"] defaults to an integer. Make it string.
df["AtomicNumber"] = df["AtomicNumber"].astype(str)

# Make "SlotOrien" and "CanBind" boolean.
df["CanBind"] = np.where(df["CanBind"] == "Y", True, False)
df["SlotOrien"] = df["SlotOrien"].map({"H":True, "V":False}) 

# Proton stack and neutron coordinates need to be float
df["PX"] = df["PX"].astype("float")
df["PY"] = df["PY"].astype("float")
df["PZ"] = df["PZ"].astype("float")
df["NX"] = df["NX"].astype("float")
df["NY"] = df["NY"].astype("float")
df["NZ"] = df["NZ"].astype("float")

# Uncomment df or df.info() to view
#df
#df.info()

In [5]:
"""
'elements'is a list of tuples used by the program to  
identify or re-select a desired atom from the gui 
dropdown or periodic table. Also used as the optional
3D atom's atomic label. 'elements' is taken from the 
df['AtomicTuple'] column.  

Starting with 1710 rows. After drop_duplicates, the 
'AtomicTuple' row index increments by 19's, 0 to 1709, 
19x expanded. The Series.array method renumbers the 
index back to the desired 0-89. The two steps are 
chained.

Next, atomicTups has been read as a list of strings. 
Those strings are converted into tuples by using 
the map and eval methods.

I found the conversion by searching "How to convert 
a list of strings into tuples in python?". It works. 
"""
atomicTups = df['AtomicTuple'].drop_duplicates().array

elements = list(map(eval, atomicTups))

#elements

In [6]:
# The periodic table's 90 buttons. Each button  
# includes: 1. Atomic type color, clist, worked 
# in steps first-fifth. 2. Atomic symbol, dlist. 
# 3. Layout width and layout heigh t

# First, the ten Atomic types, tenATypes. 
# Note. tenATypes = list(df['Atomic ...duplicates())
# works ok, the explicit written list is simpler). 
tenATypes = []
tenATypes = list(df['AtomicType'].drop_duplicates())  
#tenATypes = ['nonmetal','noble_gas','alkali_metal','alkaline_earth','metalloid',
#             'halogen','basic_metal','transition_metal','lanthanide','actinide']
#tenATypes

# Second - The 10 atomic type colors, tenColors. 
# Lined up with tenATypes
tenColors = ['skyblue','cyan','tomato','orange','springgreen','powderblue','palegreen',
         'yellow','thistle','violet']

# Third, zip together the type/color dictionary
typeColorDict = {}
typeColorDict = dict(zip(tenATypes, tenColors))
#typeColorDict

# Fourth - creating a 1710 row number/type tuple
# list which is reduced to 90 unique rows.
df['allATypes'] = list(zip(df.AtomicNumber, df.AtomicType))
ATypes = df['allATypes'].drop_duplicates().array 
#ATypes

# Fifth - Build the list of 90 colors, clist,
# used to color the Periodic table's 90 buttons.
clist = []
for i in range(90):
    clist.append(typeColorDict[ATypes[i][1]])
#clist

# The atomic symbols for the Periodic table. 
dlist = df['AtomicSymbol'].drop_duplicates().array

# Defining the buttons by adding the atomic symbols and colors to the buttons
a1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
a2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
a3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
a4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
a5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
a6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
a7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
a8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
a9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
a10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
a11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
a12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
a13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
a14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
a15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
a16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
a17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
a18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
a19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
a20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
a21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
a22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
a23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
a24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
a25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
a26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
a27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
a28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
a29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
a30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
a31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
a32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
a33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
a34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
a35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
a36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
a37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
a38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
a39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
a40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
a41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
a42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
a43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
a44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
a45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
a46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
a47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
a48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
a49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
a50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
a51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
a52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
a53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
a54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
a55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
a56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
a57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
a58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
a59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
a60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
a61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
a62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
a63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
a64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
a65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
a66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
a67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
a68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
a69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
a70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
a71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
a72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
a73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
a74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
a75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
a76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
a77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
a78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
a79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
a80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
a81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
a82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
a83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
a84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
a85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
a86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
a87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
a88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
a89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
a90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
abttns = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,
               a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,
               a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,
               a60,a61,a62,a63,a64,a65,a66,a67,a68,a69,a70,a71,a72,a73,a74,a75,a76,a77,a78,a79,
               a80,a81,a82,a83,a84,a85,a86,a87,a88,a89,a90]
# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    abttns[i].layout = widgets.Layout(border = '1px solid black') 
    abttns[i].layout.width = '100%'
    abttns[i].layout.height = '100%'  #'40px'
    i += 1
    
# Layout the 90 buttons in periodic table form
grid = GridspecLayout(10, 18)

grid[0,0] = abttns[0]
grid[0,17] = abttns[1]
grid[1,0] = abttns[2]
grid[1,1] = abttns[3]
p=12
while p < 18:
    grid[1,p] = abttns[p-8]
    p += 1
grid[2,0] = abttns[10]
grid[2,1] = abttns[11]
n=12
while n < 18:
    grid[2,n] = abttns[n]
    n += 1
m=0
while m < 18:
    grid[3,m] = abttns[18+m]
    m += 1
k=0
while k < 18:
    grid[4,k] = abttns[36+k]
    k += 1
grid[5,0] = abttns[54]
grid[5,1] = abttns[55]
# grid[5,2] = the Lanthanide series hole
i=3
while i < 18:
    grid[5,i] = abttns[68+i]
    i += 1
grid[6,0] = abttns[86]
grid[6,1] = abttns[87]
# grid[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    grid[7,j] = abttns[54+j]
    j += 1
#Actinide series
grid[8,2] = abttns[88]
grid[8,3] = abttns[89]

# The periodic table button functions
output = widgets.Output()
#display(output)
@output.capture()

def on_a1_clicked(a1): selectAtom.value = 1 
def on_a2_clicked(a2): selectAtom.value = 2 
def on_a3_clicked(a3): selectAtom.value = 3 
def on_a4_clicked(a4): selectAtom.value = 4 
def on_a5_clicked(a5): selectAtom.value = 5 
def on_a6_clicked(a6): selectAtom.value = 6 
def on_a7_clicked(a7): selectAtom.value = 7 
def on_a8_clicked(a8): selectAtom.value = 8 
def on_a9_clicked(a9): selectAtom.value = 9 
def on_a10_clicked(a10): selectAtom.value = 10 
def on_a11_clicked(a11): selectAtom.value = 11 
def on_a12_clicked(a12): selectAtom.value = 12
def on_a13_clicked(a13): selectAtom.value = 13
def on_a14_clicked(a14): selectAtom.value = 14 
def on_a15_clicked(a15): selectAtom.value = 15 
def on_a16_clicked(a16): selectAtom.value = 16 
def on_a17_clicked(a17): selectAtom.value = 17 
def on_a18_clicked(a18): selectAtom.value = 18 
def on_a19_clicked(a19): selectAtom.value = 19 
def on_a20_clicked(a20): selectAtom.value = 20 
def on_a21_clicked(a21): selectAtom.value = 21 
def on_a22_clicked(a22): selectAtom.value = 22 
def on_a23_clicked(a23): selectAtom.value = 23 
def on_a24_clicked(a24): selectAtom.value = 24
def on_a25_clicked(a25): selectAtom.value = 25 
def on_a26_clicked(a26): selectAtom.value = 26
def on_a27_clicked(a27): selectAtom.value = 27
def on_a28_clicked(a28): selectAtom.value = 28
def on_a29_clicked(a29): selectAtom.value = 29 
def on_a30_clicked(a30): selectAtom.value = 30
def on_a31_clicked(a31): selectAtom.value = 31 
def on_a32_clicked(a32): selectAtom.value = 32
def on_a33_clicked(a33): selectAtom.value = 33
def on_a34_clicked(a34): selectAtom.value = 34
def on_a35_clicked(a35): selectAtom.value = 35
def on_a36_clicked(a36): selectAtom.value = 36
def on_a37_clicked(a37): selectAtom.value = 37
def on_a38_clicked(a38): selectAtom.value = 38
def on_a39_clicked(a39): selectAtom.value = 39
def on_a40_clicked(a40): selectAtom.value = 40 
def on_a41_clicked(a41): selectAtom.value = 41 
def on_a42_clicked(a42): selectAtom.value = 42
def on_a43_clicked(a43): selectAtom.value = 43
def on_a44_clicked(a44): selectAtom.value = 44 
def on_a45_clicked(a45): selectAtom.value = 45 
def on_a46_clicked(a46): selectAtom.value = 46 
def on_a47_clicked(a47): selectAtom.value = 47 
def on_a48_clicked(a48): selectAtom.value = 48 
def on_a49_clicked(a49): selectAtom.value = 49
def on_a50_clicked(a50): selectAtom.value = 50
def on_a51_clicked(a51): selectAtom.value = 51 
def on_a52_clicked(a52): selectAtom.value = 52 
def on_a53_clicked(a53): selectAtom.value = 53 
def on_a54_clicked(a54): selectAtom.value = 54 
def on_a55_clicked(a55): selectAtom.value = 55 
def on_a56_clicked(a56): selectAtom.value = 56 
def on_a57_clicked(a57): selectAtom.value = 57 
def on_a58_clicked(a58): selectAtom.value = 58 
def on_a59_clicked(a59): selectAtom.value = 59 
def on_a60_clicked(a59): selectAtom.value = 60 
def on_a61_clicked(a61): selectAtom.value = 61 
def on_a62_clicked(a62): selectAtom.value = 62 
def on_a63_clicked(a63): selectAtom.value = 63 
def on_a64_clicked(a64): selectAtom.value = 64 
def on_a65_clicked(a65): selectAtom.value = 65 
def on_a66_clicked(a66): selectAtom.value = 66 
def on_a67_clicked(a67): selectAtom.value = 67 
def on_a68_clicked(a68): selectAtom.value = 68 
def on_a69_clicked(a69): selectAtom.value = 69 
def on_a70_clicked(a70): selectAtom.value = 70 
def on_a71_clicked(a71): selectAtom.value = 71 
def on_a72_clicked(a72): selectAtom.value = 72 
def on_a73_clicked(a73): selectAtom.value = 73 
def on_a74_clicked(a74): selectAtom.value = 74 
def on_a75_clicked(a75): selectAtom.value = 75 
def on_a76_clicked(a76): selectAtom.value = 76 
def on_a77_clicked(a77): selectAtom.value = 77 
def on_a78_clicked(a78): selectAtom.value = 78 
def on_a79_clicked(a79): selectAtom.value = 79 
def on_a80_clicked(a80): selectAtom.value = 80 
def on_a81_clicked(a81): selectAtom.value = 81 
def on_a82_clicked(a82): selectAtom.value = 82 
def on_a83_clicked(a83): selectAtom.value = 83 
def on_a84_clicked(a84): selectAtom.value = 84 
def on_a85_clicked(a85): selectAtom.value = 85 
def on_a86_clicked(a86): selectAtom.value = 86 
def on_a87_clicked(a87): selectAtom.value = 87 
def on_a88_clicked(a88): selectAtom.value = 88 
def on_a89_clicked(a89): selectAtom.value = 89 
def on_a90_clicked(a90): selectAtom.value = 90     
    
a1.on_click(on_a1_clicked)
a2.on_click(on_a2_clicked)
a3.on_click(on_a3_clicked)
a4.on_click(on_a4_clicked)
a5.on_click(on_a5_clicked)
a6.on_click(on_a6_clicked)
a7.on_click(on_a7_clicked)
a8.on_click(on_a8_clicked)
a9.on_click(on_a9_clicked)
a10.on_click(on_a10_clicked)
a11.on_click(on_a11_clicked)
a12.on_click(on_a12_clicked)
a13.on_click(on_a13_clicked)
a14.on_click(on_a14_clicked)
a15.on_click(on_a15_clicked)
a16.on_click(on_a16_clicked)
a17.on_click(on_a17_clicked)
a18.on_click(on_a18_clicked)
a19.on_click(on_a19_clicked)
a20.on_click(on_a20_clicked)
a21.on_click(on_a21_clicked)
a22.on_click(on_a22_clicked)
a23.on_click(on_a23_clicked)
a24.on_click(on_a24_clicked)
a25.on_click(on_a25_clicked)
a26.on_click(on_a26_clicked)
a27.on_click(on_a27_clicked)
a28.on_click(on_a28_clicked)
a29.on_click(on_a29_clicked)
a30.on_click(on_a30_clicked)
a31.on_click(on_a31_clicked)
a32.on_click(on_a32_clicked)
a33.on_click(on_a33_clicked)
a34.on_click(on_a34_clicked)
a35.on_click(on_a35_clicked)
a36.on_click(on_a36_clicked)
a37.on_click(on_a37_clicked)
a38.on_click(on_a38_clicked)
a39.on_click(on_a39_clicked)
a40.on_click(on_a40_clicked)
a41.on_click(on_a41_clicked)
a42.on_click(on_a42_clicked)
a43.on_click(on_a43_clicked)
a44.on_click(on_a44_clicked)
a45.on_click(on_a45_clicked)
a46.on_click(on_a46_clicked)
a47.on_click(on_a47_clicked)
a48.on_click(on_a48_clicked)
a49.on_click(on_a49_clicked)
a50.on_click(on_a50_clicked)
a51.on_click(on_a51_clicked)
a52.on_click(on_a52_clicked)
a53.on_click(on_a53_clicked)
a54.on_click(on_a54_clicked)
a55.on_click(on_a55_clicked)
a56.on_click(on_a56_clicked)
a57.on_click(on_a57_clicked)
a58.on_click(on_a58_clicked)
a59.on_click(on_a59_clicked)
a60.on_click(on_a60_clicked)
a61.on_click(on_a61_clicked)
a62.on_click(on_a62_clicked)
a63.on_click(on_a63_clicked)
a64.on_click(on_a64_clicked)
a65.on_click(on_a65_clicked)
a66.on_click(on_a66_clicked)
a67.on_click(on_a67_clicked)
a68.on_click(on_a68_clicked)
a69.on_click(on_a69_clicked)
a70.on_click(on_a70_clicked)
a71.on_click(on_a71_clicked)
a72.on_click(on_a72_clicked)
a73.on_click(on_a73_clicked)
a74.on_click(on_a74_clicked)
a75.on_click(on_a75_clicked)
a76.on_click(on_a76_clicked)
a77.on_click(on_a77_clicked)
a78.on_click(on_a78_clicked)
a79.on_click(on_a79_clicked)
a80.on_click(on_a80_clicked)
a81.on_click(on_a81_clicked)
a82.on_click(on_a82_clicked)
a83.on_click(on_a83_clicked)
a84.on_click(on_a84_clicked)
a85.on_click(on_a85_clicked)
a86.on_click(on_a86_clicked)
a87.on_click(on_a87_clicked)
a88.on_click(on_a88_clicked)
a89.on_click(on_a89_clicked)
a90.on_click(on_a90_clicked)


In [7]:
# Defining the gui widgets
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emissionType= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
coloredEmissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the equatorial(or cone) emission radius
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EmRadius',
)
# Changes the distance between adjacent orthogonal protons
ePSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 7.0,
    step= 0.1,
    description='Equ1_Pole2',
)
# Changes the distance tween top and bottom alpha parallel protons
pPSlider = widgets.FloatSlider(
    value = 4.0,
    min = 3.0,
    max = 6.0,
    step= 0.1,
    description='A width',
)
# Changes the pole to pole distance between 2 adjacent alphas
p2p3Slider = widgets.FloatSlider(
    value = 3.0,
    min = 2.0,
    max = 5.0,
    step= 0.1,
    description='A1_A2 Sep',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)

# The guiwidget control panel 
guiwidgets = widgets.VBox([selectAtom,atomsLabel,emissionType, # moved
              coloredEmissions,emRSlider,ePSlider,
              pPSlider,p2p3Slider,coneAngleSlider])

# Observe any changes in widget values, button or gui
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

ortho = ePSlider.value # proton1 equator to proton2 pole
def handle_change(change):
    global ortho
    ortho = change.new
ePSlider.observe(handle_change, 'value');

pPTP = pPSlider.value  # proton pole to proton pole 
def handle_change(change):
    global pPTP
    pPTP = change.new
pPSlider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between 2 adjacent alphas
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

cAngleDeg = coneAngleSlider.value # cone angle
def handle_change(change):
    global cAngleDeg
    cAngleDeg = change.new
coneAngleSlider.observe(handle_change, 'value')

The Periodic Table. The periodic table below is provided as a sensible way to select the desired atom, just click on it. The many atomic properties the table is known to provide may be taken into consideration while making a choice.

Most of the controls will be displayed in the cell following the periodic table. Note that within those controls, the atom can also be selected by a dropdown widget.

Other control options include: type of proton emissions displayed, color coding of the specific number of protons in each proton stack, proton separation distances, and whether to include an atomic label or not.

In [8]:
grid # Display the Periodic Table

GridspecLayout(children=(Button(description='H', layout=Layout(border='1px solid black', grid_area='widget001'…

In [10]:
guiwidgets # Display the main control widgets. 

In [11]:
# Atom data. After making an element change, 
# obtain sets of that atom's data.

# Selected Atom

An = elements[number-1]
atomicLabel = An[0] 
atomicNumber = number
elementlist = df['Element'].drop_duplicates().array
atom = elementlist[number-1]
atomsymbol = dlist[number-1]
# print(atom, atomsymbol, atomicNumber)

# Selected Atom data; slotSet_df is a dataFrame 
# containing the atom's SlotLayout details. 
# slotset_df['Protons'] column and slotlist 
# contain the number of protons in each slot.

rowstart = 19 * (atomicNumber - 1)
rowstop = rowstart + 19

slotset_df = df.iloc[rowstart:rowstop, 8:21].set_index('SlotNumber')
#slotset_df
#slotset_df.info()

slotlist = list(slotset_df.loc[:, 'Protons'])
print(slotlist)

#slotset_df
#slotset_df.info()

[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [12]:
# Create the Slotlayout Diagram. 

# The slotlist, a set of 19 integers identifying 
# the number of protons in each numbered slot is
# found in the df['AlphaType'] column, (14). and
# is identified in the previous cell.

# Using 45 non-interactive buttons to create the
# atom's slotlayout (19 + 19 + 7 + 1) diagram: 19 
# in the center SL configuration; 19 for the left
# (1,3,5,...,19) and right(2,4,6,...,18) columns;
# 7 for the slot/proton count color legend at the
# bottom; and 1 atomic symbol/mumber at the bottom 
# right. All buttons except the bottom right incl:   
# proton count color, layout width and layout height.

# The 7 colors used for color-coding each slot's 
# total proton counts, 0-6.
sclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     

sdlist = slotlist

# The colors for the 19 slots, sclist, according to
# the number of protons in each.
sclist = np.zeros(19, dtype='object')
for i in range(19):
    sclist[i] = sclrs[slotlist[i]]

# Defining the slot buttons by adding the slot position, count and colors
s1 = Button(description='1 --- '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s2 = Button(description='2 | '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s3 = Button(description='3 | '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s4 = Button(description='4 --- '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s5 = Button(description='5 --- '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s6 = Button(description='6 | '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s7 = Button(description='7 | '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s8 = Button(description='8 | '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s9 = Button(description='9 | '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s10 = Button(description='10 -- '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s11 = Button(description='11 -- '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s12 = Button(description='12 -- '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s13 = Button(description='13 -- '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s14 = Button(description='14 | '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s15 = Button(description='15 | '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s16 = Button(description='16 -- '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s17 = Button(description='17 -- '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s18 = Button(description='18 -- '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s19 = Button(description='19 -- '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# Slotlayout proton total color legend 
s20 = Button(description=str('0'), style=dict(button_color=sclrs[0]))
s21 = Button(description=str('1'), style=dict(button_color=sclrs[1]))
s22 = Button(description=str('2'), style=dict(button_color=sclrs[2]))
s23 = Button(description=str('3'), style=dict(button_color=sclrs[3]))
s24 = Button(description=str('4'), style=dict(button_color=sclrs[4]))
s25 = Button(description=str('5'), style=dict(button_color=sclrs[5]))
s26 = Button(description=str('6'), style=dict(button_color=sclrs[6]))
# Left and right slot columns, 
s27 = Button(description='1 --- '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s28 = Button(description='2 | '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s29 = Button(description='3 | '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s30 = Button(description='4 --- '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s31 = Button(description='5 --- '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s32 = Button(description='6 | '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s33 = Button(description='7 | '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s34 = Button(description='8 | '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s35 = Button(description='9 | '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s36 = Button(description='10 -- '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s37 = Button(description='11 -- '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s38 = Button(description='12 -- '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s39 = Button(description='13 -- '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s40 = Button(description='14 | '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s41 = Button(description='15 | '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s42 = Button(description='16 -- '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s43 = Button(description='17 -- '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s44 = Button(description='18 -- '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s45 = Button(description='19 -- '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# The slotlayout atom
s46 = Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0],font_weight='bold'))

# Creating the list of buttons: 19,7,19 and 1.
sbttns = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,
          s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,
          s37,s38,s39,s40,s41,s42,s43,s44,s45,s46]

# Add each button's border, layout width and height.
i=0
while i < 46 :
    sbttns[i].layout = widgets.Layout(border = '1px solid black') 
    sbttns[i].layout.width = '100%'
    sbttns[i].layout.height = '100%' 
    i += 1

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sgrid = GridspecLayout(10, 9)

sgrid[1,4] = sbttns[13] # Cr6-Elements.csv SlotLayout.
sgrid[2,4] = sbttns[3]
sgrid[2,5] = sbttns[8]
sgrid[2,6] = sbttns[12]
sgrid[3,3] = sbttns[17]
sgrid[3,4] = sbttns[1]
sgrid[3,5] = sbttns[15]
sgrid[4,2] = sbttns[9]
sgrid[4,3] = sbttns[5]
sgrid[4,4] = sbttns[0]
sgrid[4,5] = sbttns[6]
sgrid[4,6] = sbttns[11]
sgrid[5,3] = sbttns[16]
sgrid[5,4] = sbttns[2]
sgrid[5,5] = sbttns[18]
sgrid[6,2] = sbttns[10]
sgrid[6,3] = sbttns[7]
sgrid[6,4] = sbttns[4]
sgrid[7,4] = sbttns[14]
sgrid[9,1] = sbttns[19] # Color/num protons legend
sgrid[9,2] = sbttns[20] # along the bottom row
sgrid[9,3] = sbttns[21]
sgrid[9,4] = sbttns[22]
sgrid[9,5] = sbttns[23]
sgrid[9,6] = sbttns[24]
sgrid[9,7] = sbttns[25]
sgrid[0,0] = sbttns[26] # L/R Vertical slot columns
sgrid[0,8] = sbttns[27] 
sgrid[1,0] = sbttns[28]
sgrid[1,8] = sbttns[29]
sgrid[2,0] = sbttns[30]
sgrid[2,8] = sbttns[31]
sgrid[3,0] = sbttns[32]
sgrid[3,8] = sbttns[33]
sgrid[4,0] = sbttns[34]
sgrid[4,8] = sbttns[35]
sgrid[5,0] = sbttns[36]
sgrid[5,8] = sbttns[37]
sgrid[6,0] = sbttns[38]
sgrid[6,8] = sbttns[39]
sgrid[7,0] = sbttns[40]
sgrid[7,8] = sbttns[41]
sgrid[8,0] = sbttns[42]
sgrid[8,8] = sbttns[43]
sgrid[9,0] = sbttns[44]
sgrid[9,8] = sbttns[45] # Atomic symbpl and number


The Slot Layout Diagram. 

All atoms can be described by the number of protons (0-6) in each of the atom's (1-19) occupied slot positions. 

The main vertical up/down channel slots (top to bottom) are (14,4,2,1,3,5,15). There is also the front/back channel (11,17,1,16,13), and left/right channel (10,6,1,7,12); or refer to the directions from center slot 1: +/-x, +/-y, and main vertical column +/-z. The front/back/ left/right arms spin as a single group, the Carousel, around slot 1. Hydroden has a single proton in its single occupied slot 1. Helium's single slot 1 contains two protons. Hook slot positions: 16,17,18, and 19, are joined to the main up/down column at slots, 2 and 3.  

All but most of the bottom row buttons contain number pairs separated by either | or --, the vertical or horizontal characters indicate the direction of that slot's proton equatorial emission planes viewed edgewise, orthogonal to the proton's spin axis. The left side number indicates the 1-19 slot number. The right side number indicates the number of protons in that atom's slot.    

The Slot Layout is shown in the atomic configuration in the center of the diagram. It is a 3D, six pointed star projected onto the diagram's flat surface. The slot button information is repeated in the left and right side vertical columns for convienence. The seven center buttons in the bottom row contains the total protons per slot color legend. The selected atom’s symbol and atomic number are in the bottom right corner.

In [13]:
# Use the modified atom building cell to create the position 
# default dataframe.

# atomicNumber = number  # Select the desired atom from the dropdown widget or table.
grpAn = Group()  # The rotation widget spins grpAn, slots 1,4 and 5.
grpBn = Group()  # grpBn are the orthogonal top, bottom and hook slots.

if 1 <= number < 13:
    caroSecs = 2.5
if 14 <= number < 20:
    caroSecs = 4.0
if 21 <= number :
    caroSecs = 6.0

proStack1 = Group()  
proStack2 = Group()  
proStack3 = Group()
proStack4 = Group()
proStack5 = Group()
proStack6 = Group()
proStack7 = Group()
proStack8 = Group()
proStack9 = Group()
proStack10 = Group()
proStack11 = Group()
proStack12 = Group()
proStack13 = Group()
proStack14 = Group()
proStack15 = Group()
proStack16 = Group()
proStack17 = Group()
proStack18 = Group()
proStack19 = Group()

# Dimensions # pRadius is now in In[1]
nRadius = pRadius*1.125 # Neutron radius
eRadius = 2*pRadius/10 # Electron radius

# Widget Controlled variables 1. ortho, 2.pPTP, 3. p2p3Dist, 4. cAngleDeg
orthoP = ortho*pRadius
emRadius = 2*emR*pRadius # 2*orthoP - pRadius/2
coneAngle = cAngleDeg*2*piG/360
coneHeight = emRadius*tan(coneAngle)
pP1pP2 = 2*pPTP*pRadius # Widget control.
p2p3 = 2*p2p3Dist*pRadius # Widget control
n1n2 = (pP1pP2- 2*nRadius)/tan(coneAngle) # neutron-neutron orbital separation (y) inside the alpha
# e1e2 = -n1n2/10
p1p2 = pRadius*2.5  #0.75*pP1pP2 # pRadius*2.5
p1n1 = pP1pP2/2 #pP1pP2/2 #pP1pP2/2 # Alpha, up-down (z) proton-orbital neutron separation
p1n2 = pP1pP2/2 #pRadius + nRadius + 1.5*pRadius  # In line proton and neutron
p1e1 = pRadius + 0.05*pP1pP2 # Alpha, top-bottom (z) proton-electron separation
maxZNeg = 0.0 # maxZNeg and maxZPos are Used to center(z) the atom on the screen
maxZPos = 0.0

# Proton set Geometries and Materials
protonGeometry =  SphereBufferGeometry(pRadius, 24, 16)
neutronGeometry =  SphereBufferGeometry(nRadius, 24, 16)
electronGeometry =  SphereBufferGeometry(eRadius, 8, 16)
emissionGeometryP = CircleGeometry(emRadius, 24)
emissionGeometryX = ConeGeometry( emRadius, coneHeight, 16, openEnded = True )
protonMaterial = MeshStandardMaterial(color='blue')
protonMaterialB = MeshStandardMaterial(color='green')
neutronMaterial = MeshStandardMaterial(color='red')
neutronMaterialB = MeshStandardMaterial(color='green') # Color.js Find a nice color spectrum
electronMaterial = MeshStandardMaterial(color='cyan') #'orangered','salmon','green','cyan','teal','orange'
emissionmaterial7= MeshStandardMaterial(color='gray',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial1 = MeshStandardMaterial(color='magenta',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial2 = MeshStandardMaterial(color='blue',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial3 = MeshStandardMaterial(color='pink',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial4 = MeshStandardMaterial(color='salmon',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial5 = MeshStandardMaterial(color='lime',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial6 = MeshStandardMaterial(color='skyblue',side='DoubleSide',transparent = True,opacity = 0.20)

if emissionType.value == 'plane':
    emissionGeometry = emissionGeometryP
if emissionType.value == 'cones':
    emissionGeometry = emissionGeometryX
if coloredEmissions.value == True:
    emissionmaterial = emissionmaterial1
else:
    emissionmaterial = emissionmaterial7
        
def buildProtonStack(a,j): #  a is the total # of protons. j indicates the calling register
    """ The odd 'a' returns just the single, first, third, or fifth proton. The even 'a'
     returns the first, second or third alpha set, each 2 protons and 2 neutrons. """
    num = 0
    if coloredEmissions.value == True:
        if a == 1 : emissionmaterial = emissionmaterial1
        if a == 2 : emissionmaterial = emissionmaterial2
        if a == 3 : emissionmaterial = emissionmaterial3
        if a == 4 : emissionmaterial = emissionmaterial4
        if a == 5 : emissionmaterial = emissionmaterial5
        if a == 6 : emissionmaterial = emissionmaterial6
    else:
        emissionmaterial = emissionmaterial7
        # Proton z positions: # The possible formula solution, rather than the list, may be in these 2 lines 
        # protonMi.position = [0,0,a*pP1pP2/2+(a-1)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # multiAlpha solution - Not
        # protonMi.position = [0,0,(a//2)*pP1pP2/2+(a//2)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # pSet
    if a == 1 : pPositions = [0]
    if a == 2 : pPositions = [pP1pP2/2,-pP1pP2/2]
    if a == 3 : pPositions = [pP1pP2/2+p2p3/2,-pP1pP2/2+p2p3/2,-pP1pP2/2-p2p3/2]
    if a == 4 : pPositions = [pP1pP2+p2p3/2,p2p3/2,-p2p3/2,-pP1pP2-p2p3/2]
    if a == 5 : pPositions = [pP1pP2+p2p3,p2p3,0,-pP1pP2,-pP1pP2-p2p3]
    if a == 6 : pPositions = [3*pP1pP2/2+p2p3,pP1pP2/2+p2p3,pP1pP2/2,-pP1pP2/2,-pP1pP2/2-p2p3,-3*pP1pP2/2-p2p3]
    while num < a:
        protonMi = Group()
        protonMi.add(Mesh(protonGeometry, protonMaterial))
        if emissionType.value == 'plane':
            protonMi.add(Mesh(emissionGeometry, emissionmaterial))
        if emissionType.value == 'cones':
            emission1 = Object3D()
            emission1.add(Mesh(emissionGeometry, emissionmaterial))
            emission1.position = [0,0,coneHeight/2]
            emission1.rotateX(-piG/2)
            protonMi.add(emission1)
            emission2 = Object3D()
            emission2.add(Mesh(emissionGeometry, emissionmaterial))
            emission2.position = [0,0,-coneHeight/2]
            emission2.rotateX(piG/2)
            protonMi.add(emission2)
            protonMi.add(Mesh(emissionGeometryP, emissionmaterial))
        electronMi = Object3D()
        electronMi.add(Mesh(electronGeometry, electronMaterial))
        electronMi.position = [0,-n1n2/10,-p1e1]
        protonMi.add(electronMi)
        protonMi.rotateZ(random()*2*piG)
        if num%2 != 0:
            neutronPair = Group()
            neutronMi = Object3D()
            neutronMi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronMi.position = [0,-n1n2/2,-p1n1]
            neutronNi = Object3D()
            neutronNi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronNi.position = [0,n1n2/2,-p1n1]
            neutronPair.add(neutronMi)
            neutronPair.add(neutronNi)
            neutronPair.rotateZ(random()*2*piG)
            protonMi.add(neutronPair)
            protonMi.rotateX(piG)
            protonMi.rotateZ(random()*2*piG)
        protonMi.position = [0,0,pPositions[num]]
        if j == 0 : proStack1.add(protonMi)
        if j == 1 : proStack2.add(protonMi)
        if j == 2 : proStack3.add(protonMi)
        if j == 3 : proStack4.add(protonMi)
        if j == 4 : proStack5.add(protonMi)
        if j == 5 : proStack6.add(protonMi)
        if j == 6 : proStack7.add(protonMi)
        if j == 7 : proStack8.add(protonMi)
        if j == 8 : proStack9.add(protonMi)
        if j == 9 : proStack10.add(protonMi)
        if j == 10 : proStack11.add(protonMi)
        if j == 11 : proStack12.add(protonMi)
        if j == 12 : proStack13.add(protonMi)
        if j == 13 : proStack14.add(protonMi)
        if j == 14 : proStack15.add(protonMi)
        if j == 15 : proStack16.add(protonMi)
        if j == 16 : proStack17.add(protonMi)
        if j == 17 : proStack18.add(protonMi)
        if j == 18 : proStack19.add(protonMi)
        num += 1
    return

# Assemble the atomic models.
for i in range( atomicNumber ):
    
    neutronGi = Object3D()
    neutronGi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronKi = Object3D()
    neutronKi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronLi = Object3D()
    neutronLi.add(Mesh(neutronGeometry, neutronMaterial))

    if i == 0: # Slot 1 
        if 1 == atomicNumber: # Hydrogen. Slot 1 
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            slotset_df.iloc[i,8] = -p1n2            
            proStack1.position = [0, 0, 0]
            slotset_df.iloc[i,3] = 0  
            proStack1.add(neutronGi)
            grpAn.add(proStack1)
            #slotlist[i] = 1
        if 2 <= atomicNumber < 6: # Helium
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            #slotlist[i] = 2
        if 6 == atomicNumber :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            #slotlist[i] = 4
            maxZNeg = -3*pP1pP2/2-2*orthoP
            maxZPos = 3*pP1pP2/2+2*orthoP
        if 7 <= atomicNumber < 10 :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            #slotlist[i] = 6
            maxZNeg = -3*pP1pP2/2-p2p3
            maxZPos = p2p3+3*pP1pP2/2+2*orthoP
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)            
            grpAn.add(proStack1)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            #slotlist[i] = 6

    if i == 1: # Slot 2
        if 2 < atomicNumber < 6:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP]
            slotset_df.iloc[i,3] =  pP1pP2/2 + 2*orthoP            
            grpBn.add(proStack2)
            #slotlist[i] = 1
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,3] = p2p3/2+pP1pP2+2*orthoP          
            grpBn.add(proStack2)
            #slotlist[i] = 1
        if 7 <= atomicNumber < 10:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP            
            grpBn.add(proStack2)
            #slotlist[i] = 1
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP]
            slotset_df.iloc[i,3] = pP1pP2/2 + 2*orthoP 
            grpBn.add(proStack2)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP 
            grpBn.add(proStack2)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP 
            grpBn.add(proStack2)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP 
            grpBn.add(proStack2)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP 
            grpBn.add(proStack2)
            #slotlist[i] = 6

    if i == 2: # Slot 3
        if 3 < atomicNumber < 5:
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2/2 - 2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 1
        if 5 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2/2 - 2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 2
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -p2p3/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,3] = -p2p3/2-pP1pP2-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 1
        if 8 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-p2p3-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
        if 9 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-p2p3-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 2
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2/2 - 2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack3)
            #slotlist[i] = 6

    if i == 3: # Slot 4
        if 10 <= atomicNumber < 14: # Neon
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            slotset_df.iloc[i,3] = pP1pP2 + 4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 2
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            slotset_df.iloc[i,3] = pP1pP2 + 4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 2
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+4*orthoP
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack4.position = [0, 0, pP1pP2/2 + 4*orthoP]
            slotset_df.iloc[i,3] = pP1pP2/2 + 4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 1
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            slotset_df.iloc[i,3] = pP1pP2 + 4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack4.position = [0,0,p1p2/2+pP1pP2+p1p2/2+pP1pP2+4*orthoP]
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+p1p2/2+pP1pP2+4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            slotset_df.iloc[i,3] = 2*p2p3+3*pP1pP2+4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack4.position = [0,0,2*p2p3+2*pP1pP2+4*orthoP]
            slotset_df.iloc[i,3] = 2*p2p3+2*pP1pP2+4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            slotset_df.iloc[i,3] = 2*p2p3+3*pP1pP2+4*orthoP
            grpBn.add(proStack4)
            #slotlist[i] = 6

    if i == 4: # Slot 5
        if 10 <= atomicNumber < 14 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 2
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 2
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack5.position = [0, 0, -pP1pP2/2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2/2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 1
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 2
        if 21 == atomicNumber : 
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            slotset_df.iloc[i,8] = -pP1pP2/2 - p1n2 
            proStack5.add(neutronGi)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 2
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            slotset_df.iloc[i,3] = -pP1pP2 - 4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            proStack5 = Group()
            buildProtonStack(4,i)
            proStack5.position = [0,0,-p1p2/2-pP1pP2-p1p2/2-pP1pP2-4*orthoP]
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-p1p2/2-pP1pP2-4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            slotset_df.iloc[i,3] = -2*p2p3-3*pP1pP2-4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack5.position = [0,0,-2*p2p3-2*pP1pP2-4*orthoP]
            slotset_df.iloc[i,3] = -2*p2p3-2*pP1pP2-4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            slotset_df.iloc[i,3] = -2*p2p3-3*pP1pP2-4*orthoP
            grpBn.add(proStack5)
            #slotlist[i] = 6

    if i == 5: # Slot 6
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0, -2*orthoP, 0]
            slotset_df.iloc[i,2] = -2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 1
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-pP1pP2/2-2*orthoP, 0]
            slotset_df.iloc[i,2] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 2
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0, -2*orthoP, 0]
            slotset_df.iloc[i,2] = -2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 1
        if 16 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-pP1pP2/2-2*orthoP, 0]
            slotset_df.iloc[i,2] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-2*orthoP-p1p2/2-pP1pP2,0]
            slotset_df.iloc[i,2] = -2*orthoP-p1p2/2-pP1pP2
            grpAn.add(proStack6)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-p2p3-pP1pP2-2*orthoP,0]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack6.rotateX(piG/2)
            proStack6.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            grpAn.add(proStack6)
            #slotlist[i] = 6

    if i == 6: # Slot 7
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 2*orthoP, 0]
            slotset_df.iloc[i,2] = 2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 1
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,pP1pP2/2+2*orthoP, 0]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 2
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 2*orthoP, 0]
            slotset_df.iloc[i,2] = 2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 1
        if 16 <= atomicNumber < 36: 
            # Colbalt 27. Need to move neutron from non-proton slot 12 to slot 7   
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,pP1pP2/2+2*orthoP, 0]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,p1p2/2+pP1pP2+2*orthoP,0]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,p2p3+pP1pP2+2*orthoP,0]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            grpAn.add(proStack7)
            #slotlist[i] = 6 

    if i == 7: # Slot 8
        if 14 == atomicNumber:
            buildProtonStack(2,i)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [pP1pP2/2+2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = 2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [pP1pP2/2+2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [pP1pP2/2+2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, pP1pP2/2 + p1n2]
            slotset_df.iloc[i,8] = pP1pP2/2 + p1n2 
            proStack8.add(neutronGi)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [pP1pP2/2+2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack8.rotateX(-piG/2)
            proStack8.rotateY(piG/2)
            proStack8.position = [pP1pP2/2+2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack8.rotateY(piG/2)
            proStack8.position = [2*orthoP+p1p2/2+pP1pP2,0,0]
            slotset_df.iloc[i,1] = 2*orthoP+p1p2/2+pP1pP2
            grpAn.add(proStack8)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack8.rotateY(piG/2)
            proStack8.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            slotset_df.iloc[i,1] = p2p3+3*pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack8.rotateY(piG/2)
            proStack8.position = [p2p3+pP1pP2+2*orthoP,0,0] 
            slotset_df.iloc[i,1] = p2p3+pP1pP2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack8.rotateY(piG/2)
            proStack8.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            slotset_df.iloc[i,1] = p2p3+3*pP1pP2/2+2*orthoP
            grpAn.add(proStack8)
            #slotlist[i] = 6

    if i == 8: # Slot 9
        if 14 == atomicNumber: # Silicon, the carousel level
            buildProtonStack(2,i)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-pP1pP2/2-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 15 == atomicNumber:
            buildProtonStack(2,i)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-pP1pP2/2-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-pP1pP2/2-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            slotset_df.iloc[i,8] = -pP1pP2/2 - p1n2 
            proStack9.add(neutronGi)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-pP1pP2/2-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack9.rotateX(-piG/2)
            proStack9.rotateY(piG/2)
            proStack9.position = [-pP1pP2/2-2*orthoP, 0, 0]
            slotset_df.iloc[i,1] =-pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 2
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack9.rotateY(piG/2)
            proStack9.position = [-2*orthoP-p1p2/2-pP1pP2,0,0]
            slotset_df.iloc[i,1] = -2*orthoP-p1p2/2-pP1pP2
            grpAn.add(proStack9)
            #slotlist[i] = 4
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack9.rotateY(piG/2)
            proStack9.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            slotset_df.iloc[i,1] = -p2p3-3*pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 6
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack9.rotateY(piG/2)
            proStack9.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            slotset_df.iloc[i,1] = -p2p3-3*pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 5
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack9.rotateY(piG/2)
            proStack9.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            slotset_df.iloc[i,1] = -p2p3-3*pP1pP2/2-2*orthoP
            grpAn.add(proStack9)
            #slotlist[i] = 6

    if i == 9: # Slot 10
        if 21 <= atomicNumber < 29: ##################################
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            slotset_df.iloc[i,8] = -p1n2
            proStack10.add(neutronGi)
            proStack10.rotateZ(piG/2)
            proStack10.position = [0, -pP1pP2 - 4*orthoP, 0]
            slotset_df.iloc[i,2] = -pP1pP2 - 4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 1
        if 29 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack10.rotateZ(piG/2)
            proStack10.position = [0,-pP1pP2-4*orthoP,0]
            slotset_df.iloc[i,2] = -pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 2
        if 41 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack10.rotateZ(piG/2)
            proStack10.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            slotset_df.iloc[i,2] = -4*pP1pP2/2-p1p2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 1
        if 47 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack10.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            slotset_df.iloc[i,2] = -4*pP1pP2/2-p1p2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 2
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack10.rotateZ(piG/2)
            proStack10.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 1
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack10.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 2
        if 67 <= atomicNumber < 68:
            buildProtonStack(3,i)
            proStack10.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 3
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack10.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 4
        if 71 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack10.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 3
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack10.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 4
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack10.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 3
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack10.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            slotset_df.iloc[i,2] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack10)
            #slotlist[i] = 5

    if i == 10: # Slot 11
        if 22 <= atomicNumber < 27: ##################################
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            slotset_df.iloc[i,8] = -p1n2
            proStack11.add(neutronGi)
            proStack11.rotateZ(-piG/2)
            proStack11.position = [4*orthoP+pP1pP2,0,0]
            slotset_df.iloc[i,1] = 4*orthoP+pP1pP2
            grpAn.add(proStack11)
            #slotlist[i] = 1
        if 27 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack11.rotateZ(-piG/2)
            proStack11.position = [4*orthoP+pP1pP2,0,0]
            slotset_df.iloc[i,1] = 4*orthoP+pP1pP2
            grpAn.add(proStack11)
            #slotlist[i] = 2
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack11.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 4*pP1pP2/2+p1p2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 1
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack11.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 4*pP1pP2/2+p1p2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 2
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack11.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 2
        if 63 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack11.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 3
        if 71 <= atomicNumber < 74:
            buildProtonStack(2,i)
            proStack11.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 2
        if 74 <= atomicNumber < 80:
            buildProtonStack(3,i)
            proStack11.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 3
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack11.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 4
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack11.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            slotset_df.iloc[i,1] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack11)
            #slotlist[i] = 5

    if i == 11: # Slot 12
        if 21 <= atomicNumber < 27: ##################################
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            slotset_df.iloc[i,8] = -p1n2
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2 + 4*orthoP, 0]
            slotset_df.iloc[i,2] = pP1pP2 + 4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 1
            #if 27 == atomicNumber:  ############# Just a neutron goes to slot 7######
            #neutronGi.position = [0,0,0]
            #proStack12.add(neutronGi)
            #proStack12.position = [0,p1n2+pP1pP2+2*orthoP,0]
            #slotset_df.iloc[i,7] = p1n2+pP1pP2+2*orthoP
            #grpAn.add( proStack12 )
        if 28 == atomicNumber :  ##################################
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2+4*orthoP,0]
            slotset_df.iloc[i,7] = pP1pP2+4*orthoP
            grpAn.add(proStack12)
        if 29 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0,pP1pP2+4*orthoP,0]
            slotset_df.iloc[i,2] = pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 2
        if 42 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            slotset_df.iloc[i,2] = 4*pP1pP2/2+p1p2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 1
        if 47 <= atomicNumber < 54 :
            buildProtonStack(2,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            slotset_df.iloc[i,2] = 4*pP1pP2/2+p1p2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 2
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack12.position = [0,2*p2p3+2*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 1
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 2
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 3
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+2*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 4
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 2
        if 72 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 3
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 4
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 3
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            slotset_df.iloc[i,2] = 2*p2p3+3*pP1pP2+4*orthoP
            grpAn.add(proStack12)
            #slotlist[i] = 5

    if i == 12: # Slot 13  
        if 22 <= atomicNumber < 27 : 
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            slotset_df.iloc[i,8] = -p1n2
            proStack13.add(neutronGi)
            proStack13.rotateZ(-piG/2)
            proStack13.position = [-4*orthoP-pP1pP2,0,0]
            slotset_df.iloc[i,1] = -4*orthoP-pP1pP2
            grpAn.add(proStack13)
            #slotlist[i] = 1
        if 27 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack13.rotateZ(-piG/2)
            proStack13.position = [-4*orthoP-pP1pP2,0,0]
            slotset_df.iloc[i,1] = -4*orthoP-pP1pP2
            grpAn.add(proStack13)
            #slotlist[i] = 2
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack13.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -4*pP1pP2/2-p1p2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 1
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack13.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -4*pP1pP2/2-p1p2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 2
        if 57 <= atomicNumber < 63 :
            buildProtonStack(2,i)
            proStack13.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 2
        if 63 <= atomicNumber < 71 :
            buildProtonStack(3,i)
            proStack13.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-2*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 3
        if 71 == atomicNumber :
            buildProtonStack(3,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP, 0, 0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 3
        if 72 <= atomicNumber < 74 :
            buildProtonStack(2,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 2
        if 74 == atomicNumber :
            buildProtonStack(3,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 3
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 2
        if 76 <= atomicNumber < 80 :
            buildProtonStack(3,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 3
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 4
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack13.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            slotset_df.iloc[i,1] = -2*p2p3-3*pP1pP2-4*orthoP
            grpAn.add(proStack13)
            #slotlist[i] = 5

    if i == 13: # Slot 14
        if 11 <= atomicNumber  < 14 :
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0,0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
        if 17 <= atomicNumber < 18 :
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 19 <= atomicNumber < 22 :
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 23 <= atomicNumber < 26 :
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 26 <= atomicNumber < 29:
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 29 <= atomicNumber < 31:
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 37 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 48 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 55 <= atomicNumber < 57 :
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+3*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 1
        if 62 <= atomicNumber < 66 :
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+3*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 66 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = -2*p2p3-3*pP1pP2-4*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 3
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 2
        if 72 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 3
        if 78 == atomicNumber :
            buildProtonStack(4,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 4
        if 79 == atomicNumber :
            buildProtonStack(3,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 3
        if 80 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 4
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack14.rotateX(-piG/2)
            proStack14.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            slotset_df.iloc[i,3] = 3*p2p3+4.5*pP1pP2+6*orthoP
            grpBn.add(proStack14)
            #slotlist[i] = 5 

    if i == 14: # Slot 15
        if 12 == atomicNumber :
            buildProtonStack(1,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0,0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
        if 15 == atomicNumber:
            buildProtonStack(1,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
        if 17 == atomicNumber:
            buildProtonStack(1,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
        if 20 == atomicNumber:
            buildProtonStack(1,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 24 <= atomicNumber < 26:
            buildProtonStack(1,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 26 <= atomicNumber < 30 :
            buildProtonStack(2,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 2
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] =-3*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*pP1pP2/2-6*orthoP
            #slotlist[i] = 2
        if 38 == atomicNumber:
            buildProtonStack(1,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 40 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] =-3*p1p2/2-6*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 47 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 2
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 2
        if 56 == atomicNumber < 57:
            buildProtonStack(1,i)
            proStack15.rotateX(piG/2)
            proStack15.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-4.5*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 1
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP # needed
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-3*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 2
        if 63 <= atomicNumber < 66 :
            buildProtonStack(3,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-3*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 3
        if 66 == atomicNumber :
            buildProtonStack(4,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-3*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 4
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-3*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 3
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-3*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 4
        if 71 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-4.5*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 3
        if 78 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-4.5*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 4
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack15.rotateX(-piG/2)
            proStack15.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            slotset_df.iloc[i,3] = -3*p2p3-4.5*pP1pP2-6*orthoP
            grpBn.add(proStack15)
            #slotlist[i] = 5

    if i == 15: # Slot 16
        if 27 <= atomicNumber < 30:
            proStack16.add(neutronGi) ################################
            proStack16.position = [0,pP1pP2/2+p1n2,pP1pP2/2 + 2*orthoP]
            slotset_df.iloc[i,7] = pP1pP2/2+p1n2
            slotset_df.iloc[i,8] = pP1pP2/2 + 2*orthoP
            grpBn.add(proStack16)
        if 33 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack16.position = [0, pP1pP2/2+2*orthoP, pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 46 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 49 == atomicNumber:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 53 == atomicNumber:
            buildProtonStack(2,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 60 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 64 <= atomicNumber < 66:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 72 <= atomicNumber < 74:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 77 <= atomicNumber < 80:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 81 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 82 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 83 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2
        if 87 <= atomicNumber < 89 :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 1
        if 89 <= atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack16)
            #slotlist[i] = 2

    if i == 16: # Slot 17
        if 26 <= atomicNumber < 30:
            proStack17.add(neutronGi) ##################################
            proStack17.position = [0,-pP1pP2/2-p1n2,-pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,7] = -pP1pP2/2-p1n2
            slotset_df.iloc[i,8] = -pP1pP2/2 - 2*orthoP
            grpBn.add( proStack17 )
        if 34 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack17.position = [0,-pP1pP2/2 - 2*orthoP,-pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,2] = -pP1pP2/2 - 2*orthoP
            slotset_df.iloc[i,3] = -pP1pP2/2 - 2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 45 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 59 <= atomicNumber < 63 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 65 <= atomicNumber < 66 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 68 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 69 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 72 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 73 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2
        if 77 <= atomicNumber < 80 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2
        if 81 <= atomicNumber < 84:
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 84 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2
        if 87 <= atomicNumber < 90 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 1
        if 90 <= atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack17)
            #slotlist[i] = 2

    if i == 17: # Slot 18
        if 25 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP, pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 26 <= atomicNumber < 29: # A single neutron in a hook position.
            proStack18.add(neutronGi)
            proStack18.position = [0,-pP1pP2/2-p1n2,pP1pP2/2 + 2*orthoP] 
            # A single neutron in a hook position.##########################          
            slotset_df.iloc[i,7] = -pP1pP2/2-p1n2
            slotset_df.iloc[i,8] = pP1pP2/2 + 2*orthoP
            grpBn.add( proStack18 )
        if 29 <= atomicNumber < 30:
            # A two neutrons in a hook position.########################## 
            neutronGi.position = [0,-n1n2/2,0]
            proStack18.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            proStack18.add(neutronKi)
            proStack18.rotateX(piG/2)
            proStack18.position = [0,-pP1pP2/2-p1n2, pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -pP1pP2/2-p1n2
            slotset_df.iloc[i,3] = pP1pP2/2+2*orthoP
            grpBn.add( proStack18 )
        if 30 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP,pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 44 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 2
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 52 == atomicNumber :
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 2
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p1p2/2-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p1p2/2+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 57 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 2
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-pP1pP2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+pP1pP2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1           
        if 76 == atomicNumber :
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 2
        if 77 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 80 <= atomicNumber < 81 :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1
        if 85 <= atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            slotset_df.iloc[i,2] = -p2p3-3*pP1pP2/2-2*orthoP
            slotset_df.iloc[i,3] = p2p3+3*pP1pP2/2+2*orthoP
            grpBn.add(proStack18)
            #slotlist[i] = 1

    if i == 18: # Slot 19
        if 26 <= atomicNumber < 29:  ###########################
            proStack19.add(neutronGi) # A single neutron in a hook position.
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2 - 2*orthoP]
            slotset_df.iloc[i,7] = pP1pP2/2+p1n2
            slotset_df.iloc[i,8] = -pP1pP2/2 - 2*orthoP
            grpBn.add(proStack19)
        if 29 == atomicNumber : # Two neutrons ############################## 
            neutronGi.position = [0,-n1n2/2,0]
            #slotset_df.iloc[i,7] = -pP1pP2/2 - 2*orthoP
            proStack19.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            #slotset_df.iloc[i,3] = -pP1pP2/2 - 2*orthoP
            proStack19.add(neutronKi)
            proStack19.rotateX(piG/2)
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2-2*orthoP]
            #slotset_df.iloc[i,7] = pP1pP2/2+p1n2
            #slotset_df.iloc[i,8] = -pP1pP2/2-2*orthoP
            grpBn.add( proStack19 )
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
        if 32 <= atomicNumber < 35 : 
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -pP1pP2/2-2*ortho
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 35 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 43 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 52 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 53 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p1p2/2+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p1p2/2-pP1pP2-2*ortho
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 57 == atomicNumber < 58:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 58 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 63 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 66 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 68 <= atomicNumber < 71:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+pP1pP2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-pP1pP2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 76 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 2
        if 80 <= atomicNumber < 81:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 86 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1
        if 88 <= atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            slotset_df.iloc[i,2] = p2p3+3*pP1pP2/2+2*orthoP
            slotset_df.iloc[i,3] = -p2p3-3*pP1pP2/2-2*orthoP
            grpBn.add(proStack19)
            #slotlist[i] = 1

grpAn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]
grpBn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]

#print(valueL2z)
print(An) # (' 2 helium He ', 2)
print(slotlist) 

slotset_df 

(' 21 scandium Sc ', 21)
[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]


,Protons,PX,PY,PZ,SlotOrien,Neutrons,NX,NY,NZ,AlphaType,CanBind,AtomicSlotGrp
SlotNumber,,,,,,,,,,,,
1,2,0.0,0.0,0.0,True,0,0.0,0.0,0.0,1,False,21
2,2,0.0,0.0,0.7,False,0,0.0,0.0,0.0,1,False,21
3,2,0.0,0.0,-0.7,False,0,0.0,0.0,0.0,1,False,21
4,2,0.0,0.0,1.4,True,0,0.0,0.0,0.0,1,False,21
5,2,0.0,0.0,-1.4,True,1,0.0,0.0,-0.4,1,False,21
6,2,0.0,-0.7,0.0,False,0,0.0,0.0,0.0,1,False,21
7,2,0.0,0.7,0.0,False,0,0.0,0.0,0.0,1,False,21
8,2,0.7,0.0,0.0,False,1,0.0,0.0,0.4,1,False,21
9,2,-0.7,0.0,0.0,False,1,0.0,0.0,-0.4,1,False,21


In [14]:
if atomsLabel.value == True:   # Still shows UserWarning
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light,
        make_text(atomicLabel,
                  height=0.25,
                  position = [-1,1,number*0.025 + 1.5])])
else:
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light])

controller = OrbitControls(controlling=camera, screenSpacePanning=True)

renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
# initialising the scene
scene = Scene()

# Note. The UserWarning appears to be due to non JSON compliant infinite 
# values used in the pythreejs orbital camera. See the related issue. JSON 
# serialization of OrbitControls fails with jupyter_client 7.0.3,opened 
# on 22 Sep 21, https://github.com/jupyter-widgets/pythreejs/issues/366

C:\Users\Robert\anaconda3\envs\New_environ\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [15]:
# Defining the rotation widget controls for slots 1 - 19. 
spinC_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, caroSecs], values=[0, 2*piG])
spinC_clip = AnimationClip(tracks=[spinC_track])
spinC_action = AnimationAction(AnimationMixer(grpAn), spinC_clip, grpAn)

# The remaining widgets rotate the individual slots. 
spinB2_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB2_clip = AnimationClip(tracks=[spinB2_track])
spinB2_action = AnimationAction(AnimationMixer(proStack2), spinB2_clip, proStack2)

spinB3_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB3_clip = AnimationClip(tracks=[spinB3_track])
spinB3_action = AnimationAction(AnimationMixer(proStack3), spinB3_clip, proStack3)

spinB4_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB4_clip = AnimationClip(tracks=[spinB4_track])
spinB4_action = AnimationAction(AnimationMixer(proStack4), spinB4_clip, proStack4)

spinB5_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB5_clip = AnimationClip(tracks=[spinB5_track])
spinB5_action = AnimationAction(AnimationMixer(proStack5), spinB5_clip, proStack5)

spinB6_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB6_clip = AnimationClip(tracks=[spinB6_track])
spinB6_action = AnimationAction(AnimationMixer(proStack6), spinB6_clip, proStack6)

spinB7_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB7_clip = AnimationClip(tracks=[spinB7_track])
spinB7_action = AnimationAction(AnimationMixer(proStack7), spinB7_clip, proStack7)

spinB8_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB8_clip = AnimationClip(tracks=[spinB8_track])
spinB8_action = AnimationAction(AnimationMixer(proStack8), spinB8_clip, proStack8)

spinB9_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB9_clip = AnimationClip(tracks=[spinB9_track])
spinB9_action = AnimationAction(AnimationMixer(proStack9), spinB9_clip, proStack9)

spinB10_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB10_clip = AnimationClip(tracks=[spinB10_track])
spinB10_action = AnimationAction(AnimationMixer(proStack10), spinB10_clip, proStack10)

spinB11_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB11_clip = AnimationClip(tracks=[spinB11_track])
spinB11_action = AnimationAction(AnimationMixer(proStack11), spinB11_clip, proStack11)

spinB12_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB12_clip = AnimationClip(tracks=[spinB12_track])
spinB12_action = AnimationAction(AnimationMixer(proStack12), spinB12_clip, proStack12)

spinB13_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB13_clip = AnimationClip(tracks=[spinB13_track])
spinB13_action = AnimationAction(AnimationMixer(proStack13), spinB13_clip, proStack13)

spinB14_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB14_clip = AnimationClip(tracks=[spinB14_track])
spinB14_action = AnimationAction(AnimationMixer(proStack14), spinB14_clip, proStack14)

spinB15_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB15_clip = AnimationClip(tracks=[spinB15_track])
spinB15_action = AnimationAction(AnimationMixer(proStack15), spinB15_clip, proStack15)

spinB16_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB16_clip = AnimationClip(tracks=[spinB16_track])
spinB16_action = AnimationAction(AnimationMixer(proStack16), spinB16_clip, proStack16)

spinB17_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB17_clip = AnimationClip(tracks=[spinB17_track])
spinB17_action = AnimationAction(AnimationMixer(proStack17), spinB17_clip, proStack17)

spinB18_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB18_clip = AnimationClip(tracks=[spinB18_track])
spinB18_action = AnimationAction(AnimationMixer(proStack18), spinB18_clip, proStack18)

spinB19_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB19_clip = AnimationClip(tracks=[spinB19_track])
spinB19_action = AnimationAction(AnimationMixer(proStack19), spinB19_clip, proStack19)# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.

# Layout the 19 rotation controls
rgrid = GridspecLayout(8, 7)

rgrid[1, 3] = spinB6_action
rgrid[2, 3] = spinB4_action
rgrid[2, 4] = spinB11_action
rgrid[2, 5] = spinB15_action
rgrid[3, 2] = spinB18_action
rgrid[3, 3] = spinB2_action
rgrid[3, 4] = spinB16_action
rgrid[4, 1] = spinB13_action
rgrid[4, 2] = spinB9_action
rgrid[4, 3] = spinC_action
rgrid[4, 4] = spinB8_action
rgrid[4, 5] = spinB12_action
rgrid[5, 2] = spinB17_action
rgrid[5, 3] = spinB3_action
rgrid[5, 4] = spinB19_action
rgrid[6, 1] = spinB14_action
rgrid[6, 2] = spinB10_action
rgrid[6, 3] = spinB5_action
rgrid[7, 3] = spinB7_action


In [16]:
# The widget tab enclosure.

# Including a non-interactive periodic table showing the atom selected.
b1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
b2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
b3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
b4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
b5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
b6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
b7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
b8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
b9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
b10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
b11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
b12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
b13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
b14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
b15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
b16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
b17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
b18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
b19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
b20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
b21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
b22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
b23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
b24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
b25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
b26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
b27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
b28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
b29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
b30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
b31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
b32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
b33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
b34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
b35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
b36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
b37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
b38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
b39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
b40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
b41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
b42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
b43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
b44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
b45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
b46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
b47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
b48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
b49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
b50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
b51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
b52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
b53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
b54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
b55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
b56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
b57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
b58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
b59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
b60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
b61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
b62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
b63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
b64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
b65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
b66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
b67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
b68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
b69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
b70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
b71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
b72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
b73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
b74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
b75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
b76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
b77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
b78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
b79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
b80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
b81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
b82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
b83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
b84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
b85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
b86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
b87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
b88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
b89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
b90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
bbttns = [b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,b15,b16,b17,b18,b19,
               b20,b21,b22,b23,b24,b25,b26,b27,b28,b29,b30,b31,b32,b33,b34,b35,b36,b37,b38,b39,
               b40,b41,b42,b43,b44,b45,b46,b47,b48,b49,b50,b51,b52,b53,b54,b55,b56,b57,b58,b59,
               b60,b61,b62,b63,b64,b65,b66,b67,b68,b69,b70,b71,b72,b73,b74,b75,b76,b77,b78,b79,
               b80,b81,b82,b83,b84,b85,b86,b87,b88,b89,b90]

bbttns[atomicNumber-1] = Button(description=dlist[atomicNumber-1], style=dict(button_color='white',font_weight='bold'))

# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    if i == atomicNumber-1 :
        bbttns[atomicNumber-1].layout = widgets.Layout(border = '2px solid red')
    else :
        bbttns[i].layout = widgets.Layout(border = '1px solid black')
    bbttns[i].layout.width = '100%'
    bbttns[i].layout.height = '100%'  #'40px'
    i += 1
    
# Layout the 90 buttons in periodic table form
gridb = GridspecLayout(10, 18)

gridb[0,0] = bbttns[0]
gridb[0,17] = bbttns[1]
gridb[1,0] = bbttns[2]
gridb[1,1] = bbttns[3]
p=12
while p < 18:
    gridb[1,p] = bbttns[p-8]
    p += 1
gridb[2,0] = bbttns[10]
gridb[2,1] = bbttns[11]
n=12
while n < 18:
    gridb[2,n] = bbttns[n]
    n += 1
m=0
while m < 18:
    gridb[3,m] = bbttns[18+m]
    m += 1
k=0
while k < 18:
    gridb[4,k] = bbttns[36+k]
    k += 1
gridb[5,0] = bbttns[54]
gridb[5,1] = bbttns[55]
# gridb[5,2] = the Lanthanide series hole
i=3
while i < 18:
    gridb[5,i] = bbttns[68+i]
    i += 1
gridb[6,0] = bbttns[86]
gridb[6,1] = bbttns[87]
# gridb[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    gridb[7,j] = bbttns[54+j]
    j += 1
#Actinide series
gridb[8,2] = bbttns[88]
gridb[8,3] = bbttns[89]

# The tab enclosure 
tab1 = VBox(children=[renderer]) # 3D Atom
tab2 = VBox(children=[sgrid])  # Slotlayout diagram
tab3 = VBox(children=[rgrid]) # Rotation controls
tab4 = VBox(children=[gridb])  # Periodic table

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, atomicLabel)
tab.set_title(1, dlist[number-1]+' Slotlayout Diagram')
tab.set_title(2, 'Slot Rotations')
tab.set_title(3, 'Periodic Table')

VBox(children=[tab]) # Display a tab enclosure with all outputs 

In [17]:
rgrid  # Displaying another set the rotation controls allowing the 
#user to confirm rotation changes as they are made 

GridspecLayout(children=(AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', t…


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).

**SECTION 9: THE NUCLEUS** contains descriptions and diagrams 
of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project greatly relies.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'll try to figure it out.

Please pardon my understandings, personal interpretations and mistakes. Feel free to make this project better.